In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import time
from zipfile import ZipFile
import warnings
import pickle
import torch
from torch.utils.data import Dataset
import tsfel
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import lightgbm as lgb
from hiclass import LocalClassifierPerNode, LocalClassifierPerParentNode, LocalClassifierPerLevel
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings('ignore')

In [19]:
base_dir = "../logs/ensemble/base_ensemble/01_27_2025-12_43_22"

In [20]:
os.listdir(base_dir)

FileNotFoundError: [Errno 2] No such file or directory: '../logs/ensemble/base_ensemble/01_27_2025-12_30_09'

In [17]:
cv_res = pd.read_csv(os.path.join(base_dir, 'cv_report.csv'))
cv_res

,col,precision,recall,f1,support
0,Active_Power_Sensor,0.253444,0.049383,0.082659,NaN
1,Air_Flow_Sensor,0.271654,0.203540,0.232715,NaN
2,Air_Flow_Setpoint,0.145340,0.127778,0.135994,NaN
3,Air_Temperature_Sensor,0.305556,0.237208,0.267078,NaN
4,Air_Temperature_Setpoint,0.271578,0.229548,0.248801,NaN
...,...,...,...,...,...
90,Zone_Air_Dewpoint_Sensor,0.200803,0.277778,0.233100,NaN
91,Zone_Air_Humidity_Sensor,0.270175,0.237654,0.252874,NaN
92,Zone_Air_Humidity_Setpoint,0.020468,0.259259,0.037940,NaN
93,Zone_Air_Temperature_Sensor,0.295139,0.035240,0.062963,NaN


In [16]:
pd.read_csv(os.path.join(base_dir, "rf/test_predictions", "tst_preds_0.csv"))

,Alarm_0,Command_0,Parameter_0,Sensor_0,Setpoint_0,Status_0,Angle_Sensor_1,Cooling_Demand_Setpoint_1,Current_Sensor_1,Damper_Position_Setpoint_1,...,Chilled_Water_Return_Temperature_Sensor_4,Cooling_Supply_Air_Temperature_Deadband_Setpoint_4,Differential_Supply_Return_Water_Temperature_Sensor_4,Heating_Supply_Air_Temperature_Deadband_Setpoint_4,Hot_Water_Return_Temperature_Sensor_4,Low_Outside_Air_Temperature_Enable_Setpoint_4,None_4,Outside_Air_Lockout_Temperature_Setpoint_4,Peak_Power_Demand_Sensor_4,Warmest_Zone_Air_Temperature_Sensor_4
0,0.0,0.00,0.00,0.75,0.00,0.25,0.0,0.0,0.25,0.0,...,0.0,0.0,0.00,0.0,0.0,0.25,0.75,0.0,0.00,0.0
1,0.0,0.00,0.00,1.00,0.00,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,0.00,0.0,0.0,0.00,0.75,0.0,0.25,0.0
2,0.0,0.00,0.25,0.50,0.25,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,0.00,0.0,0.0,0.00,0.25,0.0,0.75,0.0
3,0.0,0.00,0.25,0.50,0.25,0.00,0.0,0.0,0.25,0.0,...,0.0,0.0,0.00,0.0,0.0,0.00,1.00,0.0,0.00,0.0
4,0.0,0.25,0.00,0.00,0.00,0.75,0.0,0.0,0.00,0.0,...,0.0,0.0,0.25,0.0,0.0,0.00,0.75,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315715,0.0,0.00,0.00,0.75,0.25,0.00,0.0,0.0,0.00,0.0,...,0.0,0.0,0.00,0.0,0.0,0.00,1.00,0.0,0.00,0.0
315716,0.0,0.00,0.00,1.00,0.00,0.00,0.0,0.0,0.50,0.0,...,0.0,0.0,0.00,0.0,0.0,0.00,1.00,0.0,0.00,0.0
315717,0.0,0.00,0.00,0.75,0.00,0.25,0.0,0.0,0.00,0.0,...,0.0,0.0,0.00,0.0,0.0,0.00,1.00,0.0,0.00,0.0
315718,0.0,0.25,0.00,0.50,0.00,0.25,0.0,0.0,0.75,0.0,...,0.0,0.0,0.25,0.0,0.0,0.00,0.75,0.0,0.00,0.0
